In [76]:
from datetime import date, time, datetime
import tables as tbl
# Not sure this is actually useful...
# import cytoolz as toolz
import sas7bdat as sas

In [276]:
sas_nbbo = sas.SAS7BDAT('./sas_data/nbbo_20110103.sas7bdat')
quotes = sas.SAS7BDAT('./sas_data/cqm_20110103.sas7bdat')

In [277]:
it = quotes.readlines()
it.next()

[u'PERMNO',
 u'group',
 u'sym_root',
 u'time_mtrail',
 u'TIME_M',
 u'EX',
 u'SYM_SUFFIX',
 u'BID',
 u'BIDSIZ',
 u'ASK',
 u'ASKSIZ',
 u'time_mroot']

In [355]:
it.next()

[10180.0,
 3.0,
 u'AKRX',
 23.0,
 datetime.time(9, 46, 37, 23000),
 u'C',
 u'',
 0.0,
 0.0,
 0.0,
 0.0,
 35197.0]

In [4]:
sas_nbbo.header

Header:
	col_count_p1: 43
	col_count_p2: 0
	column_count: 43
	compression: None
	creator: None
	creator_proc: DATASTEP
	date_created: 2015-03-20 10:13:06.208597
	date_modified: 2015-03-20 10:13:06.208597
	endianess: little
	file_type: DATA
	filename: nbbo_20110103.sas7bdat
	header_length: 8192
	lcp: 8
	lcs: 0
	mix_page_row_count: 63
	name: NBBO_20110103
	os_name: x86_64
	os_type: 2.6.32-504.1.3.e
	page_count: 382011
	page_length: 24576
	platform: unix
	row_count: 34380918
	row_length: 272
	sas_release: 9.0301M0
	server_type: Linux
	u64: True

Contents of dataset "NBBO_20110103":
Num Name          Type   Length Format Label                                               
--- ------------- ------ ------ ------ ----------------------------------------------------
  1 PERMNO        number      8        PERMNO                                              
  2 group         number      3                                                            
  3 sym_root      string     10        Securit

In [20]:
quotes.header

Header:
	col_count_p1: 12
	col_count_p2: 0
	column_count: 12
	compression: None
	creator: None
	creator_proc: DATASTEP
	date_created: 2015-03-20 10:12:41.293534
	date_modified: 2015-03-20 10:12:41.293534
	endianess: little
	file_type: DATA
	filename: cqm_20110103.sas7bdat
	header_length: 8192
	lcp: 8
	lcs: 0
	mix_page_row_count: 58
	name: CQM_20110103
	os_name: x86_64
	os_type: 2.6.32-504.1.3.e
	page_count: 326492
	page_length: 8192
	platform: unix
	row_count: 32975567
	row_length: 80
	sas_release: 9.0301M0
	server_type: Linux
	u64: True

Contents of dataset "CQM_20110103":
Num Name        Type   Length Format Label                                               
--- ----------- ------ ------ ------ ----------------------------------------------------
  1 PERMNO      number      8        PERMNO                                              
  2 group       number      3                                                            
  3 sym_root    string     10        Security symbol root  

In [249]:
PERMNO = 0
TIME_M = 4
EX = 5
BID = 7
sym_root = 2
time_mtrail = 3
time_mroot = 11

## This is our target!

In [273]:
it = iter(sas_nbbo)
it.next()
first_row = it.next()

In [274]:
# PERMNO, BBid, time_mroot, time_mtrail, TIME_M
first_row[PERMNO], first_row[sym_root], first_row[20], first_row[6], first_row[3], first_row[TIME_M]

(10180.0, u'AKRX', 6.07, 35108.0, 27.0, datetime.time(9, 45, 8, 27000))

In [275]:
second_row = it.next()
second_row[PERMNO], first_row[20]

(10180.0, 6.07)

## Some setup for our code

In [111]:
class ExchangeBids(tbl.IsDescription):
    '''Table structure for bids / exchange'''
    
    # HDF5 support for time is funny / annoying - so we don't use it
    time_mroot = tbl.Int32Col()
    time_mtrail = tbl.Int32Col()
    A = tbl.Float32Col(1, dflt=np.nan)
    B = tbl.Float32Col(1)
    C = tbl.Float32Col(1)
    D = tbl.Float32Col(1)
    I = tbl.Float32Col(1)
    J = tbl.Float32Col(1)
    K = tbl.Float32Col(1)
    M = tbl.Float32Col(1)
    N = tbl.Float32Col(1)
    T = tbl.Float32Col(1)
    P = tbl.Float32Col(1)
    S = tbl.Float32Col(1)
    Q = tbl.Float32Col(1)
    W = tbl.Float32Col(1)
    X = tbl.Float32Col(1)
    Y = tbl.Float32Col(1)
    Z = tbl.Float32Col(1)
    
def permno_row(permno):
    permno = int(permno)
    permno_table = h5f.create_table('/', 'p{}'.format(permno), ExchangeBids, 
                                    "Best bids for {}".format(permno))
    return permno_table.row

def record_time(row, time):
    row['time_mroot'] = time.hour * 24 * 60 + time.minute * 60 + time.second
    row['time_mtrail'] = time.microsecond * 1000

## Testing use of HDF5

In [67]:
h5f = tbl.open_file('test.h5', mode='w')

In [112]:
t = h5f.create_table('/', 'p126', ExchangeBids)

In [114]:
r = t.row

In [101]:
time_m = first_row[TIME_M]

In [103]:
r['time_mroot'] = time_m.hour * 24 * 60 + time_m.minute * 60 + time_m.second

In [104]:
r['time_mtrail'] = time_m.microsecond

In [118]:
r['A'] = 2.45

In [119]:
r['A']

array([ 2.45000005], dtype=float32)

In [120]:
r.append()

In [122]:
r['A']

array([ nan], dtype=float32)

In [87]:
import pandas as pd

## Done testing HDF5 - Compute the first tables!

In [259]:
tick = datetime.now()

it = quotes.readlines()
header = it.next()

curr_time = None
curr_permno = None
h5f = tbl.open_file("exchange_bids.h5", mode = "w", title = "Best bid per exchange")

for i, row in enumerate(it):
    permno = row[PERMNO]
    if curr_permno is None:
        # Our first security
        curr_permno = permno
        print permno, row[sym_root]
        out_row = permno_row(permno)
    elif curr_permno != permno:
        # We have a new security
        out_row.append()
        if permno == 0.0:
            # sas7bdat doesn't raise a StopIteration exception
            # it just starts returning empty rows
            break
        curr_permno = permno
        print permno, row[sym_root]
        curr_time = None
        out_row = permno_row(permno)
        
    if curr_time is None:
        curr_time = row[TIME_M]
        record_time(out_row, curr_time)
    elif curr_time != row[TIME_M]:
        curr_time = row[TIME_M]
        out_row.append()
        record_time(out_row, curr_time)

    
    ex = row[EX]
    if out_row[ex] == np.nan:
        out_row[ex] = row[BID]
    elif out_row[ex] < row[BID]:
        out_row[ex] = row[BID]
        
h5f.close()
tock = datetime.now()

print tock - tick

10180.0 AKRX
10201.0 AT
10258.0 CLDX
10333.0 XOMA
10838.0 CYTR
10860.0 OSUR
10952.0 CDR
11018.0 FBP
11056.0 STSA
11154.0 PLAB
11754.0 EK
11993.0 LPH
11999.0 MCP
12017.0 VRNG
12027.0 SMT
12049.0 GORO
12051.0 IL
12057.0 REE
12063.0 PARD
12090.0 TSRX
12149.0 CBRX
12169.0 DRL
12266.0 PPHM
12324.0 MY
12349.0 BODY
12359.0 NTSP
12360.0 PACB
12365.0 BITA
12389.0 TPLM
12398.0 GNOM
12409.0 SUNH
12411.0 TKMR
12413.0 ZGNX
12439.0 AVL
12444.0 DANG
12447.0 AG
12489.0 MOBI
15560.0 RSH
16505.0 BPOP
18033.0 SVNT
18148.0 SPEX
19828.0 WEN
20053.0 SNV
20670.0 CAMP
20765.0 GY
21020.0 AMR
21135.0 FOE
22323.0 ISR
22753.0 CHRS
23887.0 FTR
24441.0 CDE
29867.0 AOI
30796.0 GFI
31799.0 ELN
32540.0 ELX
32636.0 ENER
32651.0 HL
33137.0 ENZN
34666.0 NBS
34673.0 NR
35044.0 RF
37234.0 FST
39087.0 S
42024.0 OMX
42906.0 HBAN
43290.0 CYPB
43757.0 IMMU
44813.0 CVM
44951.0 SVU
46922.0 RAD
47677.0 CBB
47706.0 FSS
48267.0 LSI
48291.0 LTXC
49905.0 LIZ
49955.0 NXG
50286.0 LEE
51457.0 OSG
52337.0 THC
53613.0 MU
54148.0 PHM
57509

In [258]:
# If we interrupt the above...
h5f.close()

## Checking on results

In [260]:
h5f = tbl.open_file("exchange_bids.h5", mode = "r")

In [272]:
h5f.root.p10180[0:50]

array([ ([nan], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [6.070000171661377], [0.0], [0.0], [0.0], [0.0], 15668, 27000),
       ([nan], [0.0], [0.0], [0.0], [0.0], [6.059999942779541], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [6.070000171661377], [0.0], [0.0], [0.0], [6.059999942779541], 15668, 30000),
       ([nan], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [6.070000171661377], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], 15668, 40000),
       ([nan], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [6.070000171661377], [0.0], [0.0], [0.0], [0.0], 15668, 70000),
       ([nan], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [6.050000190734863], [0.0], [0.0], [6.070000171661377], [0.0], [0.0], [0.0], [6.039999961853027], 15668, 73000),
       ([nan], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], 15668, 83000),
       ([nan], [0.0],